# ANC StateGraph (Vertex AI Mode)

This notebook uses the same StateGraph orchestration as the Ollama notebook, with provider mode switched.


In [ ]:
#######################################################################################################

###### Dependency Preflight (Fail Fast)                                                           ######

#######################################################################################################


import importlib
import os
import sys
from pathlib import Path


def _candidate_src_paths():
    candidates = []

    # Optional explicit override.
    env_src = os.environ.get("NATURALIST_COMPANION_SRC", "").strip()
    if env_src:
        candidates.append(Path(env_src))

    # Databricks notebook context path (when available).
    try:
        notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        if "/notebooks/" in notebook_path:
            repo_workspace_path = "/Workspace" + notebook_path.split("/notebooks/", 1)[0]
            candidates.append(Path(repo_workspace_path) / "src")
    except Exception:
        pass

    cwd = Path.cwd()
    candidates.extend([
        cwd / "src",
        cwd.parent / "src",
        cwd.parent.parent / "src",
    ])

    repos_root = Path("/Workspace/Repos")
    if repos_root.exists():
        for pkg_dir in repos_root.glob("*/*/src/naturalist_companion"):
            candidates.append(pkg_dir.parent)

    deduped = []
    seen = set()
    for item in candidates:
        key = str(item)
        if key in seen:
            continue
        seen.add(key)
        deduped.append(item)
    return deduped


for src_path in _candidate_src_paths():
    if (src_path / "naturalist_companion").exists() and str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
        break


CHECKS = [('FAISS backend', ['faiss']), ('Wikipedia loader module', ['langchain_community.document_loaders', 'langchain.document_loaders']), ('Text splitter module', ['langchain_text_splitters', 'langchain.text_splitter']), ('LangChain vectorstore module', ['langchain_community.vectorstores']), ('LangChain in-memory docstore module', ['langchain_community.docstore.in_memory']), ('Vertex integration', ['langchain_google_vertexai']), ('Google AI Platform SDK', ['google.cloud.aiplatform']), ('Naturalist stategraph module', ['naturalist_companion.stategraph_shared'])]
resolved = {}
missing = []

for label, module_candidates in CHECKS:
    matched = None
    last_error = None
    for module_name in module_candidates:
        try:
            importlib.import_module(module_name)
            matched = module_name
            break
        except Exception as exc:
            last_error = f"{type(exc).__name__}: {exc}"

    if matched is not None:
        resolved[label] = matched
    else:
        missing.append((label, module_candidates, last_error))

if missing:
    lines = ["[preflight] Missing required notebook dependencies:"]
    for label, module_candidates, last_error in missing:
        lines.append(f"- {label}: expected one of {', '.join(module_candidates)}")
        if last_error:
            lines.append(f"  last error: {last_error}")

    lines.append("")
    lines.append("Run the install cell above, restart the kernel, and retry.")
    lines.append(f"Install hint: {'%pip install -q -r ../requirements-gcp-dev.txt'}")
    lines.append("If this repo is synced in Databricks, set NATURALIST_COMPANION_SRC to your repo src path if needed.")
    raise ModuleNotFoundError("\n".join(lines))

print("[preflight] Dependency check passed.")
for label, module_name in resolved.items():
    print(f"  - {label}: {module_name}")


In [ ]:
from naturalist_companion.stategraph_shared import run_i81_eval_harness, run_stategraph


In [ ]:
provider = 'vertex'
question = 'I am on I-81 near Hagerstown. What geology should I look for?'
result = run_stategraph(question, provider='vertex', config={'artifact_root': 'out/stategraph/notebook_runs'})
print(result['final_output']['provider'])
print(result['final_output']['route_decision'])
print(result['final_output']['quality'])
print(result['artifact_dir'])


In [ ]:
report = run_i81_eval_harness(provider='vertex', config={'artifact_root': 'out/stategraph/notebook_eval'})
print(report['summary'])
print(report['artifact_root'])
